# Simulate pseudo experiments using random sampling

This notebook generates new pseudo-experiments by the randomly sampling from the [sample level simulated](../Pseudomonas/Pseudomonas_sample_lvl_sim.ipynb) compendium.  The expression patterns in these new experiments are used as a negative control against the patterns in the experiments generated in [generate_E_GEOD_51409_template_experiment.ipynb](generate_E_GEOD_51409_template_experiment.ipynb) in the [differential expression analysis](DE_analysis_run.R) and [pathway enrichment analysis](find_enrichment_run.R)

In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
import ast
import pandas as pd
import numpy as np
import seaborn as sns
import random
import glob
from sklearn import preprocessing

sys.path.append("../")
from functions import utils

import warnings
warnings.filterwarnings(action='ignore')

In [2]:
# Read in config variables
config_file = os.path.abspath(os.path.join(os.getcwd(),"../configs", "config_Pa_sample_limma.tsv"))
params = utils.read_config(config_file)

In [3]:
# Load parameters
num_runs = 100
dataset_name = params["dataset_name"]
num_simulated_samples = params["num_simulated_samples"]
NN_architecture = params["NN_architecture"]
local_dir = params["local_dir"]

In [4]:
# Input files
base_dir = os.path.abspath(
  os.path.join(
      os.getcwd(), "../"))    # base dir on repo

original_data_file = os.path.join(
    local_dir,
    "pseudo_experiment",
    "Pa_compendium_02.22.2014.pcl")

simulated_data_file = os.path.join(
    local_dir,
    "experiment_simulated",
    "Pseudomonas_sample_lvl_sim",
    "Experiment_1_0.txt.xz")

mapping_file = os.path.join(
    base_dir,
    dataset_name,
    "data",
    "metadata",
    "sample_annotations.tsv")

## Process data

Notice: Originally the expression data was 0-1 normalized for use in training the VAE, however when we performed differential expression analyses we found that the normalized data had reduced variance that resulted in an inconsistency between the number of DEGs found compared to the publication. Therefore, we are re-scaling our normalized data to be in the original range of data.

In [5]:
# Read data
original_data = pd.read_table(
    original_data_file,
    header=0,
    sep='\t',
    index_col=0).T

simulated_data = pd.read_table(
    simulated_data_file,
    header=0,
    sep='\t',
    index_col=0)

print(original_data.shape)
print(simulated_data.shape)

(950, 5549)
(6000, 5549)


In [6]:
original_data.head(5)

,PA0001,PA0002,PA0003,PA0004,PA0005,PA0006,PA0007,PA0008,PA0009,PA0010,...,PA5561,PA5562,PA5563,PA5564,PA5565,PA5566,PA5567,PA5568,PA5569,PA5570
05_PA14000-4-2_5-10-07_S2.CEL,9.620090,10.575783,9.296287,9.870074,8.512268,7.903954,7.039473,10.209826,9.784684,5.485688,...,7.740609,9.730384,10.516061,10.639916,9.746849,5.768592,9.224442,11.512176,12.529719,11.804896
54375-4-05.CEL,9.327996,10.781977,9.169988,10.269239,7.237999,7.663758,6.855194,9.631573,9.404465,5.684067,...,7.127736,9.687607,10.199612,9.457152,9.318372,5.523898,7.911031,10.828271,11.597643,11.268520
AKGlu_plus_nt_7-8-09_s1.CEL,9.368599,10.596248,9.714517,9.487155,7.804147,7.681754,6.714411,9.497601,9.523126,5.766331,...,7.343241,9.717993,10.419979,10.164667,10.305005,5.806817,8.575730,10.858250,12.255953,11.309662
anaerobic_NO3_1.CEL,9.083292,9.897050,8.068471,7.310218,6.723634,7.141148,8.492302,7.740717,7.640251,5.267993,...,7.374740,8.287819,9.437053,8.936576,9.418147,5.956482,7.481406,7.687985,9.205525,9.395773
anaerobic_NO3_2.CEL,8.854901,9.931392,8.167126,7.526595,6.864015,7.154523,8.492109,7.716687,7.268094,5.427256,...,7.425398,8.588969,9.313851,8.684602,9.272818,5.729479,7.699086,7.414436,9.363494,9.424762


In [7]:
simulated_data.head(5)

,0,1,2,3,4,5,6,7,8,9,...,5539,5540,5541,5542,5543,5544,5545,5546,5547,5548
0,0.524893,0.520740,0.437727,0.503881,0.381821,0.447112,0.322690,0.448206,0.418196,0.214085,...,0.452529,0.469783,0.541290,0.466450,0.490254,0.386582,0.461289,0.511527,0.703659,0.681971
1,0.637191,0.642319,0.469714,0.620168,0.410521,0.380123,0.317894,0.591224,0.540865,0.178979,...,0.375130,0.558720,0.566576,0.531656,0.597858,0.236743,0.505707,0.538541,0.672413,0.678159
2,0.579925,0.589930,0.416886,0.573579,0.343545,0.375572,0.451579,0.498441,0.447957,0.211432,...,0.375602,0.555890,0.480001,0.472683,0.514420,0.279362,0.499572,0.444599,0.525686,0.503861
3,0.577142,0.594846,0.424863,0.555629,0.377089,0.432764,0.404817,0.489170,0.426781,0.202450,...,0.388163,0.525106,0.514754,0.489396,0.526913,0.257425,0.458029,0.440186,0.618535,0.605404
4,0.564590,0.610596,0.393297,0.505810,0.393261,0.374961,0.289844,0.525382,0.483766,0.237359,...,0.349643,0.567037,0.546622,0.510529,0.569359,0.212567,0.565946,0.432346,0.704998,0.729451


In [8]:
# 0-1 normalize per gene
scaler = preprocessing.MinMaxScaler()

original_data_scaled = scaler.fit_transform(original_data)
normalized_data = pd.DataFrame(original_data_scaled,
                                columns=original_data.columns,
                                index=original_data.index)

normalized_data.head(5)

,PA0001,PA0002,PA0003,PA0004,PA0005,PA0006,PA0007,PA0008,PA0009,PA0010,...,PA5561,PA5562,PA5563,PA5564,PA5565,PA5566,PA5567,PA5568,PA5569,PA5570
05_PA14000-4-2_5-10-07_S2.CEL,0.853357,0.725280,0.640617,0.811465,0.694460,0.533958,0.158865,0.889579,0.884945,0.176558,...,0.466871,0.702785,0.790965,0.893249,0.789939,0.164157,0.970470,0.887472,0.900484,0.880012
54375-4-05.CEL,0.778790,0.767873,0.614859,0.907865,0.398800,0.460849,0.113876,0.761351,0.801740,0.222709,...,0.352020,0.694387,0.733186,0.639074,0.681204,0.110301,0.619554,0.747656,0.749893,0.805374
AKGlu_plus_nt_7-8-09_s1.CEL,0.789155,0.729508,0.725913,0.718989,0.530160,0.466327,0.079507,0.731643,0.827707,0.241847,...,0.392405,0.700352,0.773422,0.791118,0.931585,0.172570,0.797148,0.753785,0.856253,0.811099
anaerobic_NO3_1.CEL,0.716320,0.585079,0.390211,0.193248,0.279456,0.301781,0.513547,0.342051,0.415668,0.125914,...,0.398308,0.419574,0.593955,0.527203,0.706524,0.205510,0.504767,0.105662,0.363409,0.544780
anaerobic_NO3_2.CEL,0.658015,0.592172,0.410331,0.245504,0.312028,0.305852,0.513499,0.336723,0.334226,0.162965,...,0.407801,0.478697,0.571460,0.473054,0.669643,0.155548,0.562927,0.049738,0.388931,0.548814


In [9]:
# Re-scale simulated data back into the same range as the original data
simulated_data_scaled = scaler.inverse_transform(simulated_data)

simulated_data_scaled_df = pd.DataFrame(simulated_data_scaled,
                                columns=simulated_data.columns,
                                index=simulated_data.index)

simulated_data_scaled_df.head(5)

,0,1,2,3,4,5,6,7,8,9,...,5539,5540,5541,5542,5543,5544,5545,5546,5547,5548
0,8.333439,9.585581,8.301460,8.596458,7.164818,7.618627,7.710523,8.219426,7.651806,5.646994,...,7.664078,8.543561,9.148615,8.653869,8.565930,6.779190,7.318677,9.673258,11.311484,10.381685
1,8.773331,10.174156,8.458297,9.077969,7.288515,7.398538,7.690879,8.864375,8.212359,5.496094,...,7.251056,8.996571,9.287102,8.957294,8.989948,6.098392,7.484925,9.805395,11.118085,10.354291
2,8.549008,9.920535,8.199267,8.885058,6.999852,7.383585,8.238473,8.445963,7.787804,5.635591,...,7.253579,8.982158,8.812940,8.682872,8.661157,6.292032,7.461962,9.345883,10.209929,9.101713
3,8.538107,9.944337,8.238381,8.810733,7.144423,7.571486,8.046929,8.404157,7.691038,5.596985,...,7.320603,8.825354,9.003278,8.760647,8.710387,6.192362,7.306474,9.324295,10.784611,9.831445
4,8.488939,10.020582,8.083605,8.604447,7.214126,7.381578,7.575982,8.567458,7.951437,5.747037,...,7.115051,9.038936,9.177816,8.858984,8.877645,5.988547,7.710386,9.285948,11.319766,10.722899


In [10]:
# Read in metadata
metadata = pd.read_table(
    mapping_file, 
    header=0, 
    sep='\t', 
    index_col=0)

metadata.head()

,sample_name,ml_data_source,description,nucleic_acid,medium,genotype,od,growth_setting_1,growth_setting_2,strain,temperature,treatment,additional_notes,variant_phenotype,abx_marker,biotic_int_lv_2,biotic_int_lv_1
experiment,,,,,,,,,,,,,,,,,
E-GEOD-46947,GSM1141730 1,GSM1141730_PA01_ZnO_PZO_.CEL,Pseudomonas aeruginosa PAO1 LB aerated 5 h wi...,RNA,LB,NaN,NaN,planktonic,aerated,PAO1,37.0,1 mM ZnO nanoparticles,Grown for 5h,NaN,NaN,NaN,NaN
E-GEOD-46947,GSM1141729 1,GSM1141729_PA01_none_PC_.CEL,Pseudomonas aeruginosa PAO1 LB aerated 5 h,RNA,LB,NaN,NaN,planktonic,aerated,PAO1,37.0,NaN,Grown for 5h,NaN,NaN,NaN,NaN
E-GEOD-65882,GSM1608059 1,GSM1608059_Planktonic_1.CEL,PAO1 WT. Planktonic. Rep1,RNA,PBM plus 1 g / L glucose.,WT,0.26,Planktonic,Aerated,PAO1,37.0,NaN,Grown shaking at 200rpm,NaN,NaN,NaN,NaN
E-GEOD-65882,GSM1608060 1,GSM1608060_Planktonic_2.CEL,PAO1 WT. Planktonic. Rep2,RNA,PBM plus 1 g / L glucose.,WT,0.26,Planktonic,Aerated,PAO1,37.0,NaN,Grown shaking at 200rpm,NaN,NaN,NaN,NaN
E-GEOD-65882,GSM1608061 1,GSM1608061_Planktonic_3.CEL,PAO1 WT. Planktonic. Rep3,RNA,PBM plus 1 g / L glucose.,WT,0.26,Planktonic,Aerated,PAO1,37.0,NaN,Grown shaking at 200rpm,NaN,NaN,NaN,NaN


In [11]:
map_experiment_sample = metadata[['sample_name', 'ml_data_source']]
map_experiment_sample.head()

,sample_name,ml_data_source
experiment,,
E-GEOD-46947,GSM1141730 1,GSM1141730_PA01_ZnO_PZO_.CEL
E-GEOD-46947,GSM1141729 1,GSM1141729_PA01_none_PC_.CEL
E-GEOD-65882,GSM1608059 1,GSM1608059_Planktonic_1.CEL
E-GEOD-65882,GSM1608060 1,GSM1608060_Planktonic_2.CEL
E-GEOD-65882,GSM1608061 1,GSM1608061_Planktonic_3.CEL


# Use experiment E-GEOD-51409 as a template

This experiment measures the transcriptome of *P. aeruginosa* under two different growth conditions: 28 degrees and 37 degress. This experiment contains 6 total samples with 3 samples per condition.

As a control, we will simulate a pseudo-experiment by ranomly sampling from the compendia, which does **not** preserve the experiment structure (see module ```simulate_data``` in ```functions/generate_data_parallel.py```). We will sample 6 random samples and group them into 2 groups with 3 samples per group (i.e. following the same design as the template experiment). However, this pseudo-experiment is a set of random samples that ignores experiment structure and so we don't anticipate there to find much biological signficance in this pseudo-experiment.

In [12]:
# Get experiment id
experiment_id = 'E-GEOD-51409'

In [13]:
# Get original samples associated with experiment_id
selected_mapping = map_experiment_sample.loc[experiment_id]
original_selected_sample_ids = list(selected_mapping['ml_data_source'].values)

In [14]:
# Create example random simulated
# Randomly select samples from simulated data
num_samples = len(original_selected_sample_ids)
selected_control_data = simulated_data_scaled_df.sample(n=num_samples)

# Map sample ids from original data to simulated data
selected_control_data.index = original_selected_sample_ids
selected_control_data.columns = normalized_data.columns

# Save selected samples
# This will be used as input into R script to identify differentially expressed genes
selected_control_data_file = os.path.join(
    local_dir,
    "pseudo_experiment",
    "selected_control_data_"+experiment_id+"_example.txt")

selected_control_data.to_csv(
    selected_control_data_file, float_format='%.3f', sep='\t')

In [15]:
# Create multiple control datasets
for i in range(num_runs):
    # Randomly select samples from simulated data
    num_samples = len(original_selected_sample_ids)
    selected_control_data = simulated_data_scaled_df.sample(n=num_samples)
    
    # Map sample ids from original data to simulated data
    selected_control_data.index = original_selected_sample_ids
    selected_control_data.columns = normalized_data.columns
    
    # Save selected samples
    # This will be used as input into R script to identify differentially expressed genes
    selected_control_data_file = os.path.join(
        local_dir,
        "pseudo_experiment",
        "selected_control_data_"+experiment_id+"_"+str(i)+".txt")
    
    selected_control_data.to_csv(
        selected_control_data_file, float_format='%.3f', sep='\t')